# 0 Imports

In [1]:
import numpy   as np
import pandas  as pd
import seaborn as sns

import datetime
import warnings
import inflection

from datetime             import datetime
from matplotlib           import pyplot as plt
from matplotlib.gridspec  import GridSpec
from IPython.display      import Image
from IPython.core.display import HTML

warnings.filterwarnings( 'ignore' )

## 0.1 Funções auxiliares

In [2]:
def jupyter_settings():
    %matplotlib inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [18, 10]
    plt.rcParams['font.size'] = 24
    sns.set_palette("Reds") 
    
    display(HTML( '<style>.container { width:100% !important; }</style>'))
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()

In [3]:
jupyter_settings()

## 0.2 Carregamento dos dados

In [4]:
df_raw = pd.read_csv('../inputs/data.csv', encoding= 'unicode_escape')

In [5]:
df_raw.sample(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
355360,567928,22386,JUMBO BAG PINK POLKADOT,10,9/22/2011 17:31,2.08,12380.0,Belgium
403632,571653,22065,CHRISTMAS PUDDING TRINKET POT,48,10/18/2011 12:17,0.39,14298.0,United Kingdom
107024,545400,21166,COOK WITH WINE METAL SIGN,12,3/2/2011 12:03,2.08,17048.0,United Kingdom
331309,566008,23202,JUMBO BAG VINTAGE LEAF,2,9/8/2011 12:04,2.08,16426.0,United Kingdom
86820,543601,22626,BLACK KITCHEN SCALES,2,2/10/2011 13:27,8.50,14560.0,United Kingdom
317076,564757,21003,ROSE DU SUD CUSHION COVER,1,8/30/2011 10:36,5.79,NaN,United Kingdom
388462,570428,21313,GLASS HEART T-LIGHT HOLDER,12,10/10/2011 14:02,0.85,15553.0,United Kingdom
135811,547910,21430,SET/3 RED GINGHAM ROSE STORAGE BOX,1,3/28/2011 12:23,3.75,14159.0,United Kingdom
206775,554956,22646,CERAMIC STRAWBERRY CAKE MONEY BANK,2,5/27/2011 15:17,4.13,NaN,United Kingdom
181664,552508,22403,MAGNETS PACK OF 4 VINTAGE LABELS,1,5/9/2011 16:27,2.46,NaN,United Kingdom


# 1 Descrição dos dados

In [6]:
df1 = df_raw.copy()

## 1.1 Renomeando colunas

In [7]:
snakecase = lambda x: inflection.underscore(x)

cols_new = list(map(snakecase, df1.columns))

#rename
df1.columns = cols_new

## 1.2 Dimensão dos dados

In [8]:
print('Núm de Linhas: {}'.format(df1.shape[0]))
print('Núm de Colunas: {}'.format(df1.shape[1]))

Núm de Linhas: 541909
Núm de Colunas: 8


## 1.3 Tipo dos dados

In [9]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4 Checagem de valores nulos

In [10]:
df1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5 Preenchimento de nulos

In [11]:
df1 = df1.dropna(subset=['description', 'customer_id'])
print('Percentual de dados removidos: {:.2f}%'.format((1 - (df1.shape[0] / df_raw.shape[0]))*100))

Percentual de dados removidos: 24.93%


## 1.6 Mudando tipo dos dados

In [18]:
# invoice_date
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], infer_datetime_format=True).dt.date.astype('datetime64')

# customer_id 
df1['customer_id'] = df1['customer_id'].astype('int64').astype('object')

In [19]:
df1.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
customer_id             object
country                 object
dtype: object

## 1.7 Análise descritiva

In [ ]:
num_attributes = df1.select_dtypes(include=['int64', 'float64'])
cat_attributes = df1.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

In [ ]:
num_attributes.columns

In [ ]:
cat_attributes.columns

### 1.7.1 Atributos númericos

In [ ]:
# Central Tendency - mean, mediana, 25%, 75% // dispersion - std, min, max, 
ctd1 = num_attributes.describe()

#dispersion - range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
d2 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d3 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

#join
medidas = pd.concat([ctd1, d1, d2, d3]).T.reset_index()
medidas.columns = ['attributes', 'count','mean', 'std', 'min', '25%', 'median', '75%', 'max', 'range', 'skew', 'kurtosis']


medidas

In [ ]:
plt.figure(figsize = (30,60))
num_attributes.hist(bins= 25, color= 'indianred');

### 1.7.2 Atributos categóricos

In [ ]:
cat_attributes.apply(lambda x: x.unique().shape[0])

# 2 Feature engineering

In [64]:
df2 = df1.copy()

## 2.1 Mapa Mental de hipóteses

In [ ]:
Image("../images/")

## 2.2 Criação das hipóteses

## 2.3 Lista final de hipóteses

## 2.4 Criação de novas variáveis

In [65]:
# df de referência 
df_ref = df2[['customer_id']].drop_duplicates(ignore_index=True)

# revenue = quantity * price
df2['revenue'] = df2['quantity'] * df2['unit_price']

# monetary
df_monetary = df2.groupby('customer_id')['revenue'].sum().reset_index()

# recency
df_recency = df2.groupby('customer_id')['invoice_date'].max().reset_index()
df_recency['recency_days'] = (df2['invoice_date'].max() - df_recency['invoice_date']).dt.days

# frequency
df_frequency = df2.groupby('customer_id')['invoice_no'].nunique().reset_index()

# join
c_id = 'customer_id'
df_ref = df_ref.merge(df_monetary, on=c_id, how='left').merge(df_recency, on=c_id, how='left').merge(df_frequency, on=c_id, how='left')

In [67]:
df2.head().T

,0,1,2,3,4
invoice_no,536365,536365,536365,536365,536365
stock_code,85123A,71053,84406B,84029G,84029E
description,WHITE HANGING HEART T-LIGHT HOLDER,WHITE METAL LANTERN,CREAM CUPID HEARTS COAT HANGER,KNITTED UNION FLAG HOT WATER BOTTLE,RED WOOLLY HOTTIE WHITE HEART.
quantity,6,6,8,6,6
invoice_date,2010-12-01 00:00:00,2010-12-01 00:00:00,2010-12-01 00:00:00,2010-12-01 00:00:00,2010-12-01 00:00:00
unit_price,2.55,3.39,2.75,3.39,3.39
customer_id,17850,17850,17850,17850,17850
country,United Kingdom,United Kingdom,United Kingdom,United Kingdom,United Kingdom
revenue,15.3,20.34,22.0,20.34,20.34


# 3 Filtragem dos dados

In [68]:
df3 = df_ref.copy()

## 3.1 Filtragem das linhas

## 3.2 Filtragem das colunas

# 4. Análise exploratória de dados (EDA)

In [69]:
df4 = df3.copy()

In [ ]:
num_attributes = df4.select_dtypes(include=['int64', 'float64'])
cat_attributes = df4.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

## 4.1 Análise univariada

### 4.1.1 Variável resposta

### 4.1.2 Variáveis numéricas

### 4.1.3 Variável categóricas

## 4.2 Análise bivariada

### Hn: 
**VERDADEIRA/FALSA**

### 4.2.1 Resumo das hipóteses

In [ ]:
tab =[['Hipoteses', 'Conclusao', 'Relevancia'],
      ['H1', 'Falsa', 'Baixa'],  
      ['H2', 'Falsa', 'Media'],  
      ['H3', 'Falsa', 'Media'],
      ['H4', 'Falsa', 'Baixa'],
      ['H5', '-', '-'],
      ['H6', 'Falsa', 'Baixa'],
      ['H7', 'Falsa', 'Media'],
      ['H8', 'Falsa', 'Alta'],
      ['H9', 'Falsa', 'Alta'],
      ['H10', 'Verdadeira', 'Alta'],
      ['H11', 'Verdadeira', 'Alta'],
      ['H12', 'Verdadeira', 'Baixa'],
     ]  
print(tabulate( tab, headers='firstrow'))

## 4.3 Análise multivariada

### 4.3.1 Variáveis númericas

### 4.3.2 Variáveis categóricas

# 5 Preparação dos dados

In [70]:
df5 = df4.copy()

## 5.1 Normalização

## 5.2 Rescaling

## 5.3 Transformação

### 5.3.1 Encoding

### 5.3.2 Transformação da variável resposta

### 5.3.3 Transformação de natureza

# 6 Seleção de variáveis

In [71]:
df6 = df5.copy()

## 6.1 Separação do conjunto de dados em treino e teste

## 6.2 Feature selector

## 6.3 Seleção de variáveis manual após feature selector

# 7 Otimização dos hiperparâmetros

In [72]:
df6.head()

,customer_id,revenue,invoice_date,recency_days,invoice_no
0,17850,5288.63,2011-02-10,302,35
1,13047,3079.10,2011-11-08,31,18
2,12583,7187.34,2011-12-07,2,18
3,13748,948.25,2011-09-05,95,5
4,15100,635.10,2011-01-13,330,6


In [ ]:
df7 = df6[]

## 7.1 Modelo-n

## 7.6 Comparando a perfomance dos modelos

### 7.6.1 Performance única fold

### 7.6.2 Performance cross validation

# 8 Machine learning

## 8.1 Método otimizador

## 8.2 Modelo final tunado

Tempo de execução: 1h09

# 9 Análise dos clusters

## 9.1 Performance do negócio

# 10 Deployment do modelo em produção

In [ ]:
# Save Trained Model


## 10.1 Model class

## 10.2 API handler

## 10.3 API tester